<div style="background-color:#3c3d41;padding:30px;border-radius:5px">

<h1 style="font-size:22px;color:white;margin-top:0px">
How to download MODIS L2 Active Fire Products (MYD14, MOD14) using python ?
</h1>

<p style="font-size:14px;color:white">
Author: Benjamin Marchant<br>
License: CC BY 4.0
</p>
  
</div>

In [8]:
from datetime import date
from os import path
from calendar import monthrange

import urllib.request
import urllib.request, json 
import pprint
import os

import glob
import netCDF4
import random
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import numpy.ma as ma

In [9]:
output_dir = '/Volumes/HD2/Datasets/Research/NASA'

To download a file from NASA LAADS DAAC, you need to create a token. Here are the steps to follow:

1. Visit this website: https://ladsweb.modaps.eosdis.nasa.gov/

2. Create a free account.

3. From the drop-down profile menu located in the top right corner, select "Generate Token."

4. Copy your generated token.



Please replace "the token below" with your own token. As an example, I have put in my own token. 



In [10]:
token = 'Ym1hcmNoYW50OlltVnVhbUZ0YVc0dWJXRnlZMmhoYm5SQWJtRnpZUzVuYjNZPToxNjI0NzE3MjEwOjEwZmNhNWU4ODVlNzc3OGUyMzE3NzFkZjNmNjUwMGIzNjVhMDY4ZWY'

In [11]:
opener = urllib.request.build_opener()
opener.addheaders = [('Authorization', 'Bearer {}'.format(token))]
urllib.request.install_opener(opener)

In [12]:
year = 2019
month = 8
day = 1

In [13]:
platform = 'AQUA'
instrument = 'MODIS'
product_category = 61
product = 'MYD14'

In [14]:
d0 = date(year, 1, 1)
d1 = date(year, month, day)
delta = d1 - d0

In [15]:
count_of_day = delta.days + 1

In [16]:
ladsweb_url = 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/{}/{}/{:04d}/{:03d}.json'.format(product_category,product,year,count_of_day)

print(ladsweb_url)

https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MYD14/2019/213.json


In [17]:
with urllib.request.urlopen(ladsweb_url) as url:
    data = json.loads(url.read().decode())

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(data['content'][:10])

[   {   'archiveSets': 61,
        'cksum': '2010873135',
        'dataDay': '2019-213 = 2019-08-01',
        'downloadsLink': 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MYD14/2019/213/MYD14.A2019213.0000.061.2020302175709.hdf',
        'fileId': 5691908350,
        'md5sum': '53577405052e2d994ec39f1ac2ff5ea5',
        'mtime': 1603922843,
        'name': 'MYD14.A2019213.0000.061.2020302175709.hdf',
        'products': 'MYD14',
        'resourceType': 'File',
        'self': '/archive/allData/61/MYD14/2019/213/MYD14.A2019213.0000.061.2020302175709.hdf',
        'size': 189172},
    {   'archiveSets': 61,
        'cksum': '1327693777',
        'dataDay': '2019-213 = 2019-08-01',
        'downloadsLink': 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MYD14/2019/213/MYD14.A2019213.0005.061.2020302175824.hdf',
        'fileId': 5691918818,
        'md5sum': '73ac6136ef014040ed576f0d7ca776ec',
        'mtime': 1603922962,
        'name': 'MYD14.A2019213.0005.061.

In [12]:
os.system( 'mkdir {}/{}'.format(output_dir,platform) )
os.system( 'mkdir {}/{}/{}'.format(output_dir,platform,instrument) )
os.system( 'mkdir {}/{}/{}/{}'.format(output_dir,platform,instrument,product))
os.system( 'mkdir {}/{}/{}/{}/{:04d}'.format(output_dir,platform,instrument,product,year) )
os.system( 'mkdir {}/{}/{}/{}/{:04d}/{:04d}_{:02d}_{:02d}'.format(output_dir,platform,instrument,product,year,year,month,day) )

mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS: File exists


0

In [14]:
%%time

log_error = []

for file in data['content']:
    file_name = file['name']
    #print(file_name) 
    target_dir = '{}/{}/{}/{}/{:04d}/{:04d}_{:02d}_{:02d}/{}'.format(output_dir,platform,instrument,product,year,year,month,day,file_name)
    if not path.exists( target_dir ):
        try:
            ladsweb_url = 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/{}/{}/{:04d}/{:03d}/{}'.format(product_category,product,year,count_of_day,file_name)
            urllib.request.urlretrieve(ladsweb_url,target_dir) 
        except:
            log_error.append(file)
            print('error')

CPU times: user 7.37 s, sys: 1.63 s, total: 9.01 s
Wall time: 3min 43s


In [15]:
file_list = glob.glob('{}/{}/{}/{}/{:04d}/{:04d}_{:02d}_{:02d}/*.hdf'.format(output_dir,platform,instrument,product,year,year,month,day))

len(file_list)

288

In [21]:
tot_size = 0

for file in file_list:
    file_size = os.path.getsize(file) * 0.001 # in KB
    tot_size += file_size
    print( str( round(file_size,2)  ) + ' KB')

189.17 KB
336.0 KB
214.09 KB
256.0 KB
265.63 KB
252.92 KB
236.68 KB
285.02 KB
266.24 KB
284.65 KB
503.08 KB
502.55 KB
609.51 KB
461.34 KB
228.56 KB
246.21 KB
178.26 KB
126.72 KB
199.34 KB
213.83 KB
223.14 KB
338.86 KB
196.1 KB
324.76 KB
377.19 KB
318.4 KB
286.39 KB
318.22 KB
252.03 KB
405.36 KB
597.17 KB
463.43 KB
471.33 KB
301.03 KB
177.47 KB
233.72 KB
119.79 KB
160.44 KB
220.6 KB
217.75 KB
284.19 KB
297.97 KB
270.0 KB
424.68 KB
400.6 KB
338.65 KB
293.53 KB
329.34 KB
421.26 KB
545.74 KB
550.18 KB
419.71 KB
296.52 KB
183.09 KB
147.41 KB
224.34 KB
280.98 KB
328.34 KB
343.67 KB
251.52 KB
328.86 KB
233.47 KB
269.09 KB
370.26 KB
217.24 KB
362.85 KB
325.01 KB
523.33 KB
426.21 KB
664.49 KB
415.79 KB
470.83 KB
577.06 KB
270.33 KB
211.55 KB
400.21 KB
258.09 KB
212.05 KB
321.11 KB
277.39 KB
327.25 KB
212.2 KB
253.21 KB
225.47 KB
212.81 KB
350.06 KB
444.81 KB
515.41 KB
493.45 KB
627.61 KB
476.22 KB
639.9 KB
910.41 KB
424.79 KB
257.57 KB
141.49 KB
120.74 KB
184.73 KB
302.45 KB
222.73 KB
312.87 KB

In [22]:
tot_size

98124.77599999995

In [25]:
365 * 4 * 20 / ( 60 * 24 )

20.27777777777778

In [18]:
%%time

year = 2019
month = 8

platform = 'AQUA'
instrument = 'MODIS'
product_category = 61
product = 'MYD14'

log_error = []

for day in range(1, monthrange(year, month)[1]+1):
    
    d0 = date(year, 1, 1)
    d1 = date(year, month, day)
    delta = d1 - d0

    count_of_day = delta.days + 1

    ladsweb_url = 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/{}/{}/{:04d}/{:03d}.json'.format(product_category,product,year,count_of_day)

    with urllib.request.urlopen(ladsweb_url) as url:
        data = json.loads(url.read().decode())

    os.system( 'mkdir {}/{}/{}/{}/{:04d}'.format(output_dir,platform,instrument,product,year) )
    os.system( 'mkdir {}/{}/{}/{}/{:04d}/{:04d}_{:02d}_{:02d}'.format(output_dir,platform,instrument,product,year,year,month,day) )
    
    for file in data['content']:
        file_name = file['name']
        #print(file_name) 
        target_dir = '{}/{}/{}/{}/{:04d}/{:04d}_{:02d}_{:02d}/{}'.format(output_dir,platform,instrument,product,year,year,month,day,file_name)
        if not path.exists( target_dir ):
            try:
                ladsweb_url = 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/{}/{}/{:04d}/{:03d}/{}'.format(product_category,product,year,count_of_day,file_name)
                urllib.request.urlretrieve(ladsweb_url,target_dir) 
            except:
                log_error.append(file)

mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD14/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD14/2019/2019_08_01: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD14/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD14/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD14/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD14/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD14/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD14/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD14/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD14/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD14/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD14/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS

CPU times: user 3min 53s, sys: 53.5 s, total: 4min 47s
Wall time: 1h 42min 38s
